In [3]:
import numpy as np
from sklearn import metrics
import os
from sklearn.model_selection import train_test_split
from time import time
import pandas as pd
import xgboost as xgb

In [10]:
def confusion_matrix(label, predict, n):
    """
    计算混淆矩阵
    :param label: 标签，np.array类型。形状可以是(n_sample,) 或者 (n_sample, n_classes)，当为第二种形状时可以表示多标签分类的情况
    :param predict: 预测值，与 `label` 同理
    :param n: 类别数目
    :return: 混淆矩阵，np.array类型。shape 为 (n, n)。$cm_{ij}$表示真实标签为 $i$，预测标签为 $j$ 的样本个数
    """
    k = (label >= 0) & (label < n)
    # bincount()函数用于统计数组内每个非负整数的个数
    # 详见 https://docs.scipy.org/doc/numpy/reference/generated/numpy.bincount.html
    return np.bincount(n * label[k].astype(int) + predict[k], minlength=n ** 2).reshape(n, n)


def auc(y, p, classes):
    """
    给定真实标签和预测标签，计算每个类别的auc值。实际只算出了roc曲线上一个点，即一个(fpr, tpr)，再并上(0, 0)和(1, 1)来计算auc
    :param y: 标签，np.array类型
    :param p: 预测标签，np.array类型
    :param classes: 类别，list-like，表示有哪些类别
    """
    all_aucs = np.zeros(len(classes))
    for i, c in enumerate(classes):
        _y = np.zeros_like(y)
        _y[y==c] = 1
        _y[y!=c] = 0
        _p = np.zeros_like(p)
        _p[p==c] = 1
        _p[p!=c] = 0
#         print(_y, _p)
        cm = confusion_matrix(_y, _p, 2)
#         print(cm)
        tpr = (cm[0, 0] / (cm[0, 0] + cm[0, 1])) if (cm[0, 0] + cm[0, 1]) != 0 else 0
        fpr = (cm[1, 0] / (cm[1, 0] + cm[1, 1])) if (cm[1, 0] + cm[1, 1]) != 0 else 0
        tpr = [0, tpr, 1]
        fpr = [0, fpr, 1]
        auc = metrics.auc(fpr, tpr)
        all_aucs[i] = auc
    return all_aucs

In [11]:
y = np.random.randint(0, 10, 100)
p = np.random.randint(0, 10, 100)

# 给定预测标签，计算AUC
使用OVR的策略计算每个类别的AUC
过程：
- 选择类别i作为正类，其他类别作为负类
- 将真实标签中不等于i的标记为0，等于i的标记为1
- 将预测标签中不等于i的标记为0，等于ide标记为1
- 计算混淆矩阵
- 计算(fpr, tpr)
- 计算AUC

In [12]:
classes = list(range(10))
weights = np.arange(0, 1, 0.1)
all_aucs = auc(y, p, classes)

weighted_auc = (all_aucs * weights).sum()
print(f"{all_aucs}\n{weighted_auc}")

[0.5298913  0.65555556 0.52304147 0.50747508 0.52445652 0.58219623
 0.57264957 0.46842105 0.53379416 0.50795756]
2.3789687141650595


In [13]:
classes = list(range(2))
y = np.array([0, 0, 1, 1])
p = np.array([0, 1, 0, 1])
all_aucs = auc(y, p, classes)

print(f"{all_aucs}")

[0.5 0.5]


# 加载数据
训练数据加载过程：
1. 分别加载处理好的用户特征和视频特征，以及整合的用户历史行为数据；
2. 从用户历史行为数据中筛掉在视频特征中没出现过的video_id；
3. 将行为数据中的user_id、video_id替换为对应用户/视频的特征
4. 根据不同的任务划分为`watch_label`、`is_share`的数据集

推断时，类似于上述过程拼接数据。

In [4]:
base_dir = "../2021_3_data"
test_data_dir  = os.path.join(base_dir, "testdata")
train_data_dir = os.path.join(base_dir, "traindata")

In [ ]:
user_   = np.load(os.path.join(train_data_dir, "user_features_data/user_features.npz"), allow_pickle=True)
video_  = np.load(os.path.join(train_data_dir, "video_features_data/video_features.npz"), allow_pickle=True)

In [ ]:
user_feats, user_cols = user_['features'], user_['columns']
video_feats, video_cols = video_['features'], video_['columns']

In [ ]:
user_df = pd.DataFrame(user_feats, columns=user_cols)
video_df = pd.DataFrame(video_feats, columns=video_cols)

## 加载训练数据

In [5]:
action_  = np.load(os.path.join(train_data_dir, "all_actions.npz"), allow_pickle=True) 

In [44]:
action_data, action_cols = action_['data'], action_['columns']

In [45]:
action_df = pd.DataFrame(action_data, columns=action_cols)

In [46]:
action_df.shape

(7353024, 5)

In [47]:
# 从用户历史行为数据中筛掉在视频特征中没出现过的video_id
idx1 = pd.Index(action_df['video_id'].unique())
idx2 = pd.Index(video_df['video_id'])
not_exists = idx1.difference(idx2)
not_exists

Int64Index([   15,   144,   428,   497,   876,  1174,  2127,  2199,  2334,
             3153,
            ...
            48069, 48269, 48343, 48626, 49103, 49241, 49404, 49419, 49793,
            50337],
           dtype='int64', length=243)

In [48]:
t0 = time()
n = 0
for vid in not_exists:
    tn = (action_df['video_id'] == vid).sum()
#     action_df = action_df[action_df['video_id'] != vid]
    action_df['video_id'].replace(vid, np.nan, inplace=True)
    n += tn
action_df.dropna(axis=0, inplace=True)
print(f"在视频特征中不存在的video_id在行为数据集中出现的次数 = {n}\t\t(cost {time() - t0:.3f}s)")

在视频特征中不存在的video_id在行为数据集中出现的次数 = 45006		(cost 8.837s)


In [49]:
action_df.shape

(7308018, 5)

In [52]:
video_action = video_df.merge(action_df, how='right', left_on='video_id', right_on='video_id')

In [63]:
video_action.head()

,video_id,video_name,video_score,video_duration,video_release_year,video_release_month,video_release_day,desc_0,desc_1,desc_2,...,class_9,da_0,da_1,da_2,da_3,da_4,user_id,is_watch,is_share,watch_label
0,28149,人潮汹涌,0.8,265,2021.0,2.0,12.0,0.277669,0.591679,0.016331,...,0.037483,0.075849,0.075966,0.388902,0.383616,0.075666,4239342,1,0,2
1,115,数码宝贝：最后的进化,0.81,194,2020.0,10.0,30.0,0.026072,0.026047,0.026042,...,0.626961,0.084394,0.361578,0.082888,0.082888,0.388253,3577036,1,0,0
2,3636,东海异闻录,0.77,3807,2021.0,4.0,24.0,0.018332,0.351056,0.018328,...,0.041772,0.324848,0.083544,0.083544,0.083544,0.424519,5527504,1,0,5
3,12968,刺杀小说家,0.79,7791,2021.0,2.0,12.0,0.022649,0.022656,0.022645,...,0.034261,0.50428,0.072258,0.070551,0.069007,0.283903,1117889,1,0,0
4,860,飞驰人生,0.84,5889,2019.0,2.0,5.0,0.345659,0.029107,0.029101,...,0.037924,0.076261,0.39165,0.077193,0.075847,0.379048,1117889,1,0,4


In [67]:
user_video_action = user_df.merge(video_action, how='right', left_on='user_id', right_on='user_id')

In [68]:
# 删除 video_name 列，调整video_id列的顺序
user_video_action.drop(['video_name', 'is_watch'], axis=1, inplace=True)
user_video_action.insert(1, 'video_id', user_video_action.pop('video_id'))
user_video_action.head(3)

,user_id,video_id,age_0,age_1,age_2,age_3,age_4,age_5,age_6,age_7,...,class_7,class_8,class_9,da_0,da_1,da_2,da_3,da_4,is_share,watch_label
0,4239342.0,28149,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.037486,0.395894,0.037483,0.075849,0.075966,0.388902,0.383616,0.075666,0,2
1,3577036.0,115,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.041444,0.041444,0.626961,0.084394,0.361578,0.082888,0.082888,0.388253,0,0
2,5527504.0,3636,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.041779,0.04178,0.041772,0.324848,0.083544,0.083544,0.083544,0.424519,0,5


In [97]:
# 保存数据以便下次直接读取
np.savez(os.path.join(train_data_dir, "train"), data=user_video_action.values, columns=user_video_action.columns.tolist())

In [ ]:
# 删除 video_id、user_id列
user_video_action.drop(['user_id', 'video_id'], axis=1, inplace=True)

In [70]:
dataset = user_video_action

## 加载测试数据

In [88]:
test_df = pd.read_csv(os.path.join(test_data_dir, "test.csv"))

In [90]:
# 测试数据集中存在video_id没有在视频特征中出现
idx1 = pd.Index(test_df['video_id'].unique())
idx2 = pd.Index(video_df['video_id'].unique())
non_exists = idx1.difference(idx2)
non_exists

Int64Index([   15,   144,   428,   497,   876,  1174,  1589,  1906,  2127,
             2199,
            ...
            47945, 48069, 48269, 48343, 48626, 49241, 49404, 49419, 49793,
            50337],
           dtype='int64', length=276)

In [91]:
video_test = video_df.merge(test_df, how='right', left_on='video_id', right_on='video_id')
user_video_test = user_df.merge(video_test, how='right', left_on='user_id', right_on='user_id')

In [94]:
user_video_test.insert(1, 'video_id', user_video_test.pop('video_id'))

In [96]:
# 保存数据以便下次直接读取
np.savez(os.path.join(test_data_dir, "test"), data=user_video_test.values, columns=user_video_test.columns.tolist())

In [ ]:
inference_dataset = user_video_test

# XGBoost

In [ ]:
def train_xgb(X_train, y_train, params):
    xg_train = xgb.DMatrix(X_train.values, label=y_train.values, enable_categorical=True)
    xg_test = xgb.DMatrix(X_test.values, label=y_test.values, enable_categorical=True)
    
    watchlist = [(xg_train, 'train'), (xg_test, 'test')]
    num_round = 5
    # train xgb
    bst = xgb.train(_param, xg_train, num_round, watchlist)
    # get prediction
    pred = bst.predict(xg_test)
    error_rate = np.sum(pred != y_test) / test_y.shape[0]
    print('Test error using softmax = {}'.format(error_rate))

    # do the same thing again, but output probabilities
    param['objective'] = 'multi:softprob'
    bst = xgb.train(param, xg_train, num_round, watchlist)
    # Note: this convention has been changed since xgboost-unity
    # get prediction, this is in 1D array, need reshape to (ndata, nclass)
    pred_prob = bst.predict(xg_test).reshape(y_test.shape[0], 6)
    pred_label = np.argmax(pred_prob, axis=1)
    error_rate = np.sum(pred_label != y_test) / y_test.shape[0]
    print('Test error using softprob = {}'.format(error_rate))


In [79]:
# 准备数据
watch_label = dataset.pop('watch_label')
is_share = dataset.pop('is_share')
train_idx, test_idx = train_test_split(dataset.index, test_size=0.2, random_state=0)

In [80]:
X_train = dataset.iloc[train_idx]
X_test  = dataset.iloc[test_idx]

In [82]:
X_train.shape, X_test.shape

((5846414, 72), (1461604, 72))

## watch_label 预测

In [81]:
y_train = watch_label.iloc[train_idx]
y_test  = watch_label.iloc[test_idx]

In [99]:
t0 = time()
xg_train = xgb.DMatrix(X_train.values, label=y_train.values, enable_categorical=True)
xg_test = xgb.DMatrix(X_test.values, label=y_test.values, enable_categorical=True)
# setup parameters for xgboost
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['eta'] = 0.1
param['max_depth'] = 6
param['nthread'] = 4
param['num_class'] = 10
watchlist = [(xg_train, 'train'), (xg_test, 'test')]
print(f"Data preparing finished ...\t\t({time()-t0:.3f}s)")

Data preparing finished ...		(48.264s)


In [ ]:
num_round = 20
t0 = time()
wl_bst_sm = xgb.train(param, xg_train, num_round, watchlist)
print(f"{num_round}-rounds Training finished ...\t\t({time()-t0:.3f}s)")

[22:28:52] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softmax' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	train-mlogloss:2.07532	test-mlogloss:2.07479


In [ ]:
# get prediction
pred = wl_bst_sm.predict(xg_test)
error_rate = np.sum(pred != y_test) / y_test.shape[0]
print('Test error using softmax = {}'.format(error_rate))

In [ ]:
# do the same thing again, but output probabilities
num_round = 10
param['objective'] = 'multi:softprob'
wl_bst_sp = xgb.train(param, xg_train, num_round, watchlist)

In [ ]:
# Note: this convention has been changed since xgboost-unity
# get prediction, this is in 1D array, need reshape to (ndata, nclass)
pred_prob = wl_bst_sp.predict(xg_test).reshape(y_test.shape[0], 6)
pred_label = np.argmax(pred_prob, axis=1)
error_rate = np.sum(pred_label != y_test) / y_test.shape[0]
print('Test error using softprob = {}'.format(error_rate))

## is_share 预测

In [ ]:
y_train = is_share.iloc[train_idx]
y_test  = is_share.iloc[test_idx]

In [ ]:
xg_train = xgb.DMatrix(X_train.values, label=y_train.values, enable_categorical=True)
xg_test = xgb.DMatrix(X_test.values, label=y_test.values, enable_categorical=True)
# setup parameters for xgboost
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['eta'] = 0.1
param['max_depth'] = 6
param['nthread'] = 4
param['num_class'] = 10

watchlist = [(xg_train, 'train'), (xg_test, 'test')]


In [ ]:
num_round = 5
sh_bst = xgb.train(param, xg_train, num_round, watchlist)

# 服务器间同步文件

## 推向Digix服务器

In [2]:
!scp ./models.ipynb digix@49.123.120.71:/home/digix/digix/Models/models.ipynb 

models.ipynb                                  100% 4972     2.9MB/s   00:00    


In [16]:
!scp ../explore-data.ipynb digix@49.123.120.71:/home/digix/digix/explore-data.ipynb 

explore-data.ipynb                            100%  280KB  10.6MB/s   00:00    


## 从Digix服务器拉数据

In [43]:
!scp  digix@49.123.120.71:/home/digix/digix/Models/LightGBM.ipynb ./LightGBM.ipynb

LightGBM.ipynb                                100%   71KB   8.7MB/s   00:00    


In [44]:
!scp  digix@49.123.120.71:/home/digix/digix/Models/feature_engineering.ipynb ./feature_engineering.ipynb

feature_engineering.ipynb                     100%  274KB  10.5MB/s   00:00    
